In [23]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import numpy as np


### Load Pre-Trained VGG network
> #### https://github.com/chenyaofo
> ### Note: There is some issue with using Batch Norm before ReLu as it creates a bias in the network. So people exchange the order between the two for tackling the bias

In [125]:
### First time when you wan to download the model
device=torch.device('cuda')
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg11_bn", pretrained=True)
model=model.to(device)
### For future uses:Loading from the local

# model_1=torch.hub.load("C:/Users/nmura/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master",'hubconf.py',source='local')



Using cache found in C:\Users\nmura/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


### Check which pre-trained model are available:

In [3]:
from pprint import pprint
pprint(torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True))

Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to C:\Users\nmura/.cache\torch\hub\master.zip


['cifar100_mobilenetv2_x0_5',
 'cifar100_mobilenetv2_x0_75',
 'cifar100_mobilenetv2_x1_0',
 'cifar100_mobilenetv2_x1_4',
 'cifar100_repvgg_a0',
 'cifar100_repvgg_a1',
 'cifar100_repvgg_a2',
 'cifar100_resnet20',
 'cifar100_resnet32',
 'cifar100_resnet44',
 'cifar100_resnet56',
 'cifar100_shufflenetv2_x0_5',
 'cifar100_shufflenetv2_x1_0',
 'cifar100_shufflenetv2_x1_5',
 'cifar100_shufflenetv2_x2_0',
 'cifar100_vgg11_bn',
 'cifar100_vgg13_bn',
 'cifar100_vgg16_bn',
 'cifar100_vgg19_bn',
 'cifar100_vit_b16',
 'cifar100_vit_b32',
 'cifar100_vit_h14',
 'cifar100_vit_l16',
 'cifar100_vit_l32',
 'cifar10_mobilenetv2_x0_5',
 'cifar10_mobilenetv2_x0_75',
 'cifar10_mobilenetv2_x1_0',
 'cifar10_mobilenetv2_x1_4',
 'cifar10_repvgg_a0',
 'cifar10_repvgg_a1',
 'cifar10_repvgg_a2',
 'cifar10_resnet20',
 'cifar10_resnet32',
 'cifar10_resnet44',
 'cifar10_resnet56',
 'cifar10_shufflenetv2_x0_5',
 'cifar10_shufflenetv2_x1_0',
 'cifar10_shufflenetv2_x1_5',
 'cifar10_shufflenetv2_x2_0',
 'cifar10_vgg11_bn

### Downlaoding the Dataset and Creating the Dataloaders

In [99]:
### Transformation 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
### Pytorch Datasets
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)
### Dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [100]:
len(train_dataset)

50000

### Create the Forget Dataset

In [101]:
train_split_dataset,forget_split_dataset=torch.utils.data.random_split(train_dataset,lengths=[45000,5000])

#### Forget and Retain

In [102]:
retain_dataloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=128, shuffle=True, num_workers=2)
forget_dataloader=torch.utils.data.DataLoader(forget_split_dataset, batch_size=128, shuffle=True, num_workers=2)

### Stochastic Network

In [103]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.normal(m.weight,mean=0,std=20)
        m.bias.data.fill_(0.)
    elif isinstance(m, nn.Conv2d):
        torch.nn.init.normal(m.weight,mean=0,std=20)

In [126]:
import copy
model_random=copy.deepcopy(model)
model_random=model_random.apply(init_weights)
model_random.to(device)

C:\Users\nmura\AppData\Local\Temp\ipykernel_19560\250970810.py:6: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight,mean=0,std=20)
C:\Users\nmura\AppData\Local\Temp\ipykernel_19560\250970810.py:3: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight,mean=0,std=20)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

### Objective Function: 
> ### Criterion: KL divergence Loss

In [106]:
def KL_div(output:np.array,truth:np.array):
    """""
    output: 2D array
    truth: 2D array

    """""
    soft=torch.nn.Softmax(dim=1)

    ### Applying the siftmax to the outputs of both the networks
    output=soft(output)
    truth=soft(truth)
    
    return torch.sum(truth*(torch.log(truth)-torch.log(output))),output.size(0)


criterion=nn.KLDivLoss(reduction="batchmean")

### Optimizer

In [107]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  

#### Impair Step

In [127]:
num_epochs=10

for i in range(num_epochs):

    num_corr=0
    num_items=0
    for batch_idx,(inputs,targets) in enumerate(forget_dataloader):

        inputs, targets = inputs.to(device), targets.to(device)
        #### Original Model
        out=model(inputs)

        ## random model
        with torch.no_grad():
            model_random.eval()
            out_random=model_random(inputs)
            
        ###Zero Gradient
        optimizer.zero_grad()
        ### Loss of the model
        soft=torch.nn.Softmax(dim=1)

         ### Applying the siftmax to the outputs of both the networks
        output=soft(out)
        truth=soft(out_random)

        loss=criterion(output,truth)
        
        ### Backward Propogation
        loss.backward()

        ### Optimization step
        optimizer.step()

        ###number of correct items
        num_corr+=torch.sum(torch.argmax(output,axis=1)==targets)
        # print(batch_idx,num_corr)
        # print(targets,torch.argmax(output,axis=1))
        num_items+=targets.size(0)
    
        

    print('epochs',i,num_corr/num_items,num_corr)



    





epochs 0 tensor(0.9928, device='cuda:0') tensor(4964, device='cuda:0')
epochs 1 tensor(0.9924, device='cuda:0') tensor(4962, device='cuda:0')
epochs 2 tensor(0.9942, device='cuda:0') tensor(4971, device='cuda:0')
epochs 3 tensor(0.9912, device='cuda:0') tensor(4956, device='cuda:0')
epochs 4 tensor(0.9928, device='cuda:0') tensor(4964, device='cuda:0')
epochs 5 tensor(0.9920, device='cuda:0') tensor(4960, device='cuda:0')
epochs 6 tensor(0.9920, device='cuda:0') tensor(4960, device='cuda:0')
epochs 7 tensor(0.9940, device='cuda:0') tensor(4970, device='cuda:0')
epochs 8 tensor(0.9934, device='cuda:0') tensor(4967, device='cuda:0')
epochs 9 tensor(0.9942, device='cuda:0') tensor(4971, device='cuda:0')


### Evaluate the pre-trained model on CIFAR-10

In [78]:
### Push the model to GPU
device=torch.device('cuda')
model_random=model_random.to(device)
# output=model.forward(torch.FloatTensor(test_data[b'data'].numpy()).reshape(-1,3,32,32))

In [77]:
with torch.no_grad():
    model_random.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        out=model_random(inputs)
        print(torch.argmax(out,axis=1))
        print('actual labels',targets)
        
        if batch_idx==3:
            break


tensor([2, 2, 0, 2, 4, 2, 4, 4, 0, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2,
        2, 2, 2, 9, 2, 4, 4, 0, 2, 9, 2, 4, 2, 0, 2, 2, 2, 4, 4, 7, 2, 4, 4, 4,
        4, 4, 2, 2, 4, 4, 4, 2, 2, 4, 0, 4, 2, 4, 4, 2, 2, 4, 2, 2, 3, 4, 0, 4,
        2, 2, 2, 4, 9, 2, 4, 2, 2, 2, 4, 0, 2, 4, 2, 2, 2, 2, 4, 4, 0, 2, 2, 4,
        4, 0, 2, 2, 0, 2, 4, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2,
        2, 4, 4, 4, 2, 0, 4, 0], device='cuda:0')
actual labels tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7,
        8, 3, 1, 2, 8, 0, 8, 3], device='cuda:0')
tensor([2, 3, 7, 4, 4, 0, 0, 9, 2, 2, 4, 2, 2, 2, 0, 2, 2, 2, 4, 4, 2, 4, 2, 2,
      

In [76]:
out

tensor([[ 1.8442e+31, -5.0307e+30,  2.1097e+31,  ...,  1.1683e+31,
          1.2616e+31,  1.1677e+31],
        [ 4.9849e+31, -3.4924e+31,  4.3136e+31,  ...,  1.7350e+31,
          8.5682e+30,  4.2503e+31],
        [ 1.5537e+31, -1.9391e+31,  2.5463e+31,  ...,  4.4038e+30,
          1.0957e+31,  2.6654e+31],
        ...,
        [ 7.9146e+30, -1.0102e+31,  2.3317e+31,  ...,  2.3273e+31,
          1.3055e+31,  1.0298e+31],
        [ 2.4701e+31, -2.6977e+31,  1.9855e+31,  ...,  2.3883e+31,
          9.3079e+30,  1.8130e+31],
        [ 1.3003e+31, -1.2094e+31,  1.7972e+31,  ...,  1.8652e+31,
          3.4659e+30,  2.0752e+31]], device='cuda:0')

In [21]:
targets

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7,
        8, 3, 1, 2, 8, 0, 8, 3], device='cuda:0')

In [135]:
x=torch.FloatTensor(test_data[b'data'].numpy()).reshape(-1,32,32,3)
### normalisation

var=x.reshape(10000,-1,3)
var_min=var.min(1,keepdim=True)[0]
var_min=var_min.unsqueeze(1)

var_max=var.max(1,keepdim=True)[0]
var_max=var_max.unsqueeze(1)


x=(x-var_min)/(x-var_max)
### correct input order
x=x.permute(0,3,1,2)
## standardise
x=torchvision.transforms.functional.normalize(x,mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010])
x=x.to(device)

In [12]:
inputs.shape

torch.Size([128, 3, 224, 224])

In [137]:
with torch.no_grad():
    model.eval()
    out=model.forward(x[0:300])

In [74]:
out.shape

torch.Size([300, 10])

In [138]:
torch.argmax(out,axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [107]:
test_data[b'labels'][0:30]

[3,
 8,
 8,
 0,
 6,
 6,
 1,
 6,
 3,
 1,
 0,
 9,
 5,
 7,
 9,
 8,
 5,
 7,
 8,
 6,
 7,
 0,
 4,
 9,
 5,
 2,
 4,
 0,
 9,
 6]

In [31]:

torch.cuda.empty_cache()